In [ ]:
#https://www.kaggle.com/code/duythanhng/birdclef-2022-keras-simple-tutorial
# python 3

In [36]:
import os
import json 
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model, Sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Concatenate

In [6]:
#!pip install tensorflow

In [37]:
train_meta = pd.read_csv("train_metadata.csv")
train_meta

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,time,url,filename
0,afrsil1,[],"['call', 'flight call']",12.3910,-1.4930,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,2.5,08:00,https://www.xeno-canto.org/125458,afrsil1/XC125458.ogg
1,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],19.8801,-155.7254,Euodice cantans,African Silverbill,Dan Lane,Creative Commons Attribution-NonCommercial-Sha...,3.5,08:30,https://www.xeno-canto.org/175522,afrsil1/XC175522.ogg
2,afrsil1,[],"['call', 'song']",16.2901,-16.0321,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:30,https://www.xeno-canto.org/177993,afrsil1/XC177993.ogg
3,afrsil1,[],"['alarm call', 'call']",17.0922,54.2958,Euodice cantans,African Silverbill,Oscar Campbell,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:00,https://www.xeno-canto.org/205893,afrsil1/XC205893.ogg
4,afrsil1,[],['flight call'],21.4581,-157.7252,Euodice cantans,African Silverbill,Ross Gallardy,Creative Commons Attribution-NonCommercial-Sha...,3.0,16:30,https://www.xeno-canto.org/207431,afrsil1/XC207431.ogg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14847,zebdov,[],"['adult', 'song']",22.6499,120.2872,Geopelia striata,Zebra Dove,Jerome Chie-Jen Ko,Creative Commons Attribution-NonCommercial-Sha...,2.5,08:36,https://www.xeno-canto.org/629769,zebdov/XC629769.ogg
14848,zebdov,[],"['adult', 'sex uncertain', 'song']",1.4168,103.7287,Geopelia striata,Zebra Dove,Kim Chuah Lim,Creative Commons Attribution-NonCommercial-Sha...,4.0,08:00,https://www.xeno-canto.org/642415,zebdov/XC642415.ogg
14849,zebdov,[],"['adult', 'call', 'sex uncertain']",1.4347,103.7043,Geopelia striata,Zebra Dove,Kim Chuah Lim,Creative Commons Attribution-NonCommercial-Sha...,4.0,09:00,https://www.xeno-canto.org/665873,zebdov/XC665873.ogg
14850,zebdov,[],['song'],3.3508,101.2451,Geopelia striata,Zebra Dove,Jelle Scharringa,Creative Commons Attribution-NonCommercial-Sha...,2.5,11:15,https://www.xeno-canto.org/666194,zebdov/XC666194.ogg


In [38]:
labels = list(train_meta['primary_label'].unique())
print(labels)

['afrsil1', 'akekee', 'akepa1', 'akiapo', 'akikik', 'amewig', 'aniani', 'apapan', 'arcter', 'barpet', 'bcnher', 'belkin1', 'bkbplo', 'bknsti', 'bkwpet', 'blkfra', 'blknod', 'bongul', 'brant', 'brnboo', 'brnnod', 'brnowl', 'brtcur', 'bubsan', 'buffle', 'bulpet', 'burpar', 'buwtea', 'cacgoo1', 'calqua', 'cangoo', 'canvas', 'caster1', 'categr', 'chbsan', 'chemun', 'chukar', 'cintea', 'comgal1', 'commyn', 'compea', 'comsan', 'comwax', 'coopet', 'crehon', 'dunlin', 'elepai', 'ercfra', 'eurwig', 'fragul', 'gadwal', 'gamqua', 'glwgul', 'gnwtea', 'golphe', 'grbher3', 'grefri', 'gresca', 'gryfra', 'gwfgoo', 'hawama', 'hawcoo', 'hawcre', 'hawgoo', 'hawhaw', 'hawpet1', 'hoomer', 'houfin', 'houspa', 'hudgod', 'iiwi', 'incter1', 'jabwar', 'japqua', 'kalphe', 'kauama', 'laugul', 'layalb', 'lcspet', 'leasan', 'leater1', 'lessca', 'lesyel', 'lobdow', 'lotjae', 'madpet', 'magpet1', 'mallar3', 'masboo', 'mauala', 'maupar', 'merlin', 'mitpar', 'moudov', 'norcar', 'norhar2', 'normoc', 'norpin', 'norsho', 

In [40]:
import soundfile as sf
import os

def cutAudio(file_path, is_save):
    # First load the file
    filename = file_path.replace("/", "_")
    file_path = "train_audio/" + file_path
    audio, sr = librosa.load(file_path)

    # Get number of samples for 5 seconds; replace 5 by any number
    buffer = 5 * sr

    samples_total = len(audio)
    samples_wrote = 0
    counter = 1

    audio_split = []
    audio_filenames = []
    while samples_wrote < samples_total:
        #check if the buffer is not exceeding total samples 
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote

        block = audio[samples_wrote : (samples_wrote + buffer)]
        audio_split.append(block)

        # Write 5 second segment
        if is_save == True:
            out_filename = "kaggle/working/each5s/split_" + str(counter) + "_" + filename
            audio_filenames.append(out_filename)
            sf.write(out_filename, block, sr)
        counter += 1
        samples_wrote += buffer
    return audio_split, sr, audio_filenames

In [41]:
def splitTrainAudio(_df):
    data = []
    for index, row in _df.iterrows():
        cutAudio(row["filename"], True)
        audio_lst, sr, filenames = cutAudio(row["filename"], True)
        for idx, y in enumerate(audio_lst):
            data.append([row["primary_label"], row["filename"], filenames[idx]])

    data_df = pd.DataFrame(data, columns=['primary_label', 'original_filename', 'filename'])
    data_df.to_csv("kaggle/working/data_df.csv", index=False)

In [42]:
# Sample Data
data_frames = []
for label in labels:
    tmp_df = train_meta[train_meta["primary_label"] == label].sample(n=1, replace=True).reset_index(drop=True)
    data_frames.append(tmp_df)
sample_df = pd.concat(data_frames).reset_index(drop=True)
sample_df

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,time,url,filename
0,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],19.8801,-155.7254,Euodice cantans,African Silverbill,Dan Lane,Creative Commons Attribution-NonCommercial-Sha...,3.5,08:30,https://www.xeno-canto.org/175522,afrsil1/XC175522.ogg
1,akekee,['iiwi'],['call'],22.1467,-159.6134,Loxops caeruleirostris,Akekee,Daniel Lane,Creative Commons Attribution-NonCommercial-Sha...,3.5,12:30,https://www.xeno-canto.org/27306,akekee/XC27306.ogg
2,akepa1,"['akiapo', 'apapan', 'hawama', 'hawcre', 'iiwi']",['call'],19.6292,-155.3597,Loxops coccineus,Hawaii Akepa,Brooks Rownd,Creative Commons Attribution-NonCommercial-Sha...,2.5,14:14,https://www.xeno-canto.org/122473,akepa1/XC122473.ogg
3,akiapo,"['akepa1', 'apapan', 'hawcre', 'omao', 'warwhe1']","['begging call', 'mechanical tapping']",19.5866,-155.3402,Hemignathus wilsoni,Akiapolaau,Brooks Rownd,Creative Commons Attribution-NonCommercial-Sha...,3.5,16:56,https://www.xeno-canto.org/147003,akiapo/XC147003.ogg
4,akikik,[],['call'],22.0815,-159.5550,Oreomystis bairdi,Akikiki,Patrick Blake,Creative Commons Attribution-NonCommercial-Sha...,3.5,14:00,https://www.xeno-canto.org/317034,akikik/XC317034.ogg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,whttro,[],['call'],-18.8391,-159.7812,Phaethon lepturus,White-tailed Tropicbird,Matthias Feuersenger,Creative Commons Attribution-NonCommercial-Sha...,4.0,12:28,https://www.xeno-canto.org/110890,whttro/XC110890.ogg
148,wiltur,[],"['male', 'song']",44.8432,-93.3698,Meleagris gallopavo,Wild Turkey,Jonathon Jongsma,Creative Commons Attribution-ShareAlike 3.0,4.5,06:06,https://www.xeno-canto.org/134155,wiltur/XC134155.ogg
149,yebcar,[],['call'],-32.8606,-60.6486,Paroaria capitata,Yellow-billed Cardinal,Franco Vushurovich,Creative Commons Attribution-NonCommercial-Sha...,4.5,08:30,https://www.xeno-canto.org/573571,yebcar/XC573571.ogg
150,yefcan,['reccar'],['song'],21.2637,-157.8194,Crithagra mozambica,Yellow-fronted Canary,Dan Lane,Creative Commons Attribution-NonCommercial-Sha...,2.5,09:00,https://www.xeno-canto.org/174823,yefcan/XC174823.ogg


In [45]:

splitTrainAudio(sample_df)

In [46]:
data_df = pd.read_csv("kaggle/working/data_df.csv")
data_df

,primary_label,original_filename,filename
0,afrsil1,afrsil1/XC175522.ogg,kaggle/working/each5s/split_1_afrsil1_XC175522...
1,afrsil1,afrsil1/XC175522.ogg,kaggle/working/each5s/split_2_afrsil1_XC175522...
2,afrsil1,afrsil1/XC175522.ogg,kaggle/working/each5s/split_3_afrsil1_XC175522...
3,afrsil1,afrsil1/XC175522.ogg,kaggle/working/each5s/split_4_afrsil1_XC175522...
4,afrsil1,afrsil1/XC175522.ogg,kaggle/working/each5s/split_5_afrsil1_XC175522...
...,...,...,...
1408,yefcan,yefcan/XC174823.ogg,kaggle/working/each5s/split_3_yefcan_XC174823.ogg
1409,zebdov,zebdov/XC573140.ogg,kaggle/working/each5s/split_1_zebdov_XC573140.ogg
1410,zebdov,zebdov/XC573140.ogg,kaggle/working/each5s/split_2_zebdov_XC573140.ogg
1411,zebdov,zebdov/XC573140.ogg,kaggle/working/each5s/split_3_zebdov_XC573140.ogg
